This notebook pre-processes designs from the [photonics-opt-testbed](https://github.com/NanoComp/photonics-opt-testbed/tree/main/RGB_metalens) repo, e.g. by trimming whitespace and adjusting the `.csv` file format.

In [ ]:
import glob

import matplotlib.pyplot as plt
import numpy as onp


def process_reference_design(fname, overwrite):
    design = onp.genfromtxt(fname, delimiter=",")

    # The `Ex` designs have solid on the substrate side, and void on the ambient side.
    # The `Ey` designs are completely surrounded by void.
    design_type = fname.split("/")[-2]
    assert design_type in ("Ex", "Ez")
    pad_value_cap = 0
    pad_value_base = 0 if design_type == "Ez" else 1

    lo = 0
    for i in range(design.shape[1]):
        if not onp.all(design[:, i] == pad_value_base):
            lo = i
            break

    hi = design.shape[1]
    for i in range(design.shape[1], -1, -1):
        if not onp.all(design[:, i - 1] == pad_value_cap):
            hi = i
            break

    assert lo == 0 or onp.all(design[:, lo - 1] == pad_value_base)
    assert hi == design.shape[1] - 1 or onp.all(design[:, hi + 1] == pad_value_cap)
    cropped_design = design[:, lo:hi]

    # Special treatment for the `Ex/Rasmus70nm.csv` design, which includes a void
    # column of pixels at the base (adjacent to the substrate). We assume this is
    # an error and perform additional cropping, so that the lens is connected to the
    # substrate and not floating above it.
    if fname.endswith("Ex/Rasmus70nm.csv"):
        if design.shape == (1022, 122):
            # Ensure additional cropping is only done for original unmodified design.
            assert onp.all(cropped_design[:, 0] == 0)
            assert onp.all(cropped_design[10:-10, 1:10] == 1)
            cropped_design = cropped_design[:, 10:]

    cropped_design = onp.pad(
        cropped_design, ((0, 0), (1, 0)), constant_values=pad_value_base
    )
    cropped_design = onp.pad(
        cropped_design, ((0, 0), (0, 1)), constant_values=pad_value_cap
    )

    if overwrite:
        onp.savetxt(fname, cropped_design, fmt="%.2f", delimiter=",")

    return cropped_design, design


fnames = glob.glob("*/*.csv")
plt.figure(figsize=(18, 1.5 * len(fnames)))
for i, fname in enumerate(fnames):
    # Set `overwrite=True` to modify the original designs. Once modified, the designs
    # should remain unchanged by additional calls to `process_reference_design`.
    design, original_design = process_reference_design(fname, overwrite=False)
    ax = plt.subplot(len(fnames), 2, 2 * i + 1)
    im = ax.imshow(onp.rot90(original_design), cmap="gray")
    ax.axis(False)

    ax = plt.subplot(len(fnames), 2, 2 * i + 2)
    im = ax.imshow(onp.rot90(design), cmap="gray")
    ax.axis(False)